In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import shap
import mlflow
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import os
import sys

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from config import PROCESSED_DATA_PATH, MLFLOW_TRACKING_URI, MODEL_NAME

# --- Set up MLflow CORRECTLY ---
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("ETF_Trend_Prediction")

# --- Load Data ---
data = pd.read_parquet(PROCESSED_DATA_PATH)

# Separate features (X) and target (y)
X = data.drop('target', axis=1)
y = data['target']

C:\Users\dawso\Dev\Personal\AIGrind\mlops-etf-forecasting\.venv\Lib\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


2025/08/31 14:05:15 INFO mlflow.tracking.fluent: Experiment with name 'ETF_Trend_Prediction' does not exist. Creating a new experiment.


In [2]:
# Define the chronological split point
# For example, use data up to the end of 2021 for training, and 2022 onwards for testing.
train_end_date = '2021-12-31'
test_start_date = '2022-01-01'

X_train = X.loc[:train_end_date]
y_train = y.loc[:train_end_date]

X_test = X.loc[test_start_date:]
y_test = y.loc[test_start_date:]

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")

mlflow.set_experiment("ETF_Trend_Prediction")

Training set size: 2380
Test set size: 908


<Experiment: artifact_location='file:C:\\Users\\dawso\\Dev\\Personal\\AIGrind\\mlops-etf-forecasting\\mlruns/492371556750088850', creation_time=1756663515713, experiment_id='492371556750088850', last_update_time=1756663515713, lifecycle_stage='active', name='ETF_Trend_Prediction', tags={}>

In [3]:
# Train Logistic Regression
with mlflow.start_run(run_name="LogisticRegression_Baseline"):
    model_lr = LogisticRegression(max_iter=1000, random_state=42)
    model_lr.fit(X_train, y_train)
    y_pred_lr = model_lr.predict(X_test)
    
    # Log metrics
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred_lr))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred_lr))
    print(f"Logistic Regression F1 Score: {f1_score(y_test, y_pred_lr):.4f}")

# Train Random Forest
with mlflow.start_run(run_name="RandomForest_Baseline"):
    model_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    model_rf.fit(X_train, y_train)
    y_pred_rf = model_rf.predict(X_test)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred_rf))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred_rf))
    print(f"Random Forest F1 Score: {f1_score(y_test, y_pred_rf):.4f}")

Logistic Regression F1 Score: 0.6968


Random Forest F1 Score: 0.6290


In [ ]:
def objective(trial):
    # Define the search space for hyperparameters
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42
    }
    
    model = xgb.XGBClassifier(**params)
    
    # Use TimeSeriesSplit for robust cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    score = cross_val_score(model, X_train, y_train, cv=tscv, scoring='f1', n_jobs=1).mean()
    
    return score

In [5]:
# Run the study to find the best params
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=125) 

best_params = study.best_params
print("Best XGBoost Params:", best_params)

# Train the final XGBoost model with the best parameters and log to MLflow
with mlflow.start_run(run_name="XGBoost_Tuned_Champion") as run:
    print("--- Training and Logging Champion XGBoost Model ---")
    best_params = study.best_params
    
    # Log hyperparameters
    mlflow.log_params(best_params)
    
    # Train the model
    model_xgb = xgb.XGBClassifier(**best_params, random_state=42)
    model_xgb.fit(X_train, y_train)
    
    # Evaluate and log metrics
    y_pred_xgb = model_xgb.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_xgb)
    f1 = f1_score(y_test, y_pred_xgb)
    roc_auc = roc_auc_score(y_test, model_xgb.predict_proba(X_test)[:, 1])
    
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("roc_auc", roc_auc)
    
    print(f"Champion XGBoost F1 Score: {f1:.4f}")

    # Log the model itself
    mlflow.xgboost.log_model(
        xgb_model=model_xgb,
        artifact_path="xgb-model",
        registered_model_name="etf-xgboost-predictor" # This also registers the model
    )
    print("Champion model logged and registered.")

    # --- NEW CODE TO ADD STARTS HERE ---
    
    print("--- Logging SHAP assets for the dashboard ---")
    import joblib # Make sure joblib is imported
    
    # 1. Create and save the SHAP explainer object
    explainer = shap.TreeExplainer(model_xgb)
    joblib.dump(explainer, "explainer.joblib")
    
    # 2. Save the X_test dataframe needed for plotting
    X_test.to_parquet("X_test.parquet")
    
    # 3. Log these files as MLflow artifacts in specific sub-folders
    mlflow.log_artifact("explainer.joblib", artifact_path="shap_explainer")
    mlflow.log_artifact("X_test.parquet", artifact_path="shap_xtest")
    
    print("SHAP explainer and X_test data successfully logged as artifacts.")

[I 2025-08-31 14:05:16,414] A new study created in memory with name: no-name-2c938972-51bc-48bc-906b-e0e29be072d8


[I 2025-08-31 14:05:19,127] Trial 0 finished with value: 0.5360648099528504 and parameters: {'n_estimators': 913, 'max_depth': 9, 'learning_rate': 0.268623433264274, 'subsample': 0.6970252053947417, 'colsample_bytree': 0.7860761957433005, 'gamma': 2.9927657451165164}. Best is trial 0 with value: 0.5360648099528504.


[I 2025-08-31 14:05:21,022] Trial 1 finished with value: 0.5273697647558737 and parameters: {'n_estimators': 969, 'max_depth': 6, 'learning_rate': 0.26896643435380124, 'subsample': 0.7911558053027973, 'colsample_bytree': 0.5234654285441125, 'gamma': 0.6665220179909237}. Best is trial 0 with value: 0.5360648099528504.


[I 2025-08-31 14:05:24,477] Trial 2 finished with value: 0.5136679077501333 and parameters: {'n_estimators': 501, 'max_depth': 8, 'learning_rate': 0.016080635025102645, 'subsample': 0.9000955317950565, 'colsample_bytree': 0.8984698273219339, 'gamma': 1.477836729875634}. Best is trial 0 with value: 0.5360648099528504.


[I 2025-08-31 14:05:26,526] Trial 3 finished with value: 0.499456117817623 and parameters: {'n_estimators': 359, 'max_depth': 5, 'learning_rate': 0.03974017879621874, 'subsample': 0.9432716782847458, 'colsample_bytree': 0.9646054177029721, 'gamma': 0.1120877927400421}. Best is trial 0 with value: 0.5360648099528504.


...

Champion XGBoost F1 Score: 0.6112


C:\Users\dawso\Dev\Personal\AIGrind\mlops-etf-forecasting\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:06:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


Successfully registered model 'etf-xgboost-predictor'.
Created version '1' of model 'etf-xgboost-predictor'.


Champion model logged and registered.
--- Logging SHAP assets for the dashboard ---


SHAP explainer and X_test data successfully logged as artifacts.


In [6]:
# --- MLP Challenger Model ---
# Step 1: Imports and Data Scaling
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

# Deep learning models are sensitive to feature scale. We must standardize our data.
# We fit the scaler ONLY on the training data to prevent data leakage from the test set.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Data successfully scaled.")
print(f"Shape of scaled training data: {X_train_scaled.shape}")

Data successfully scaled.
Shape of scaled training data: (2380, 32)


In [7]:
# Convert numpy arrays to PyTorch Tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders to handle batching
# We don't shuffle time-series data to preserve temporal order if needed, 
# but for a simple MLP, shuffling is often acceptable. Let's keep it False for rigor.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("PyTorch Tensors and DataLoaders created.")

PyTorch Tensors and DataLoaders created.


In [8]:
# Step 2: Define the MLP Architecture
class ETF_MLP(nn.Module):
    def __init__(self, input_size, hidden_size_1=128, hidden_size_2=64, dropout_rate=0.5):
        """
        Initializes the MLP model.
        
        Args:
            input_size (int): The number of input features.
            hidden_size_1 (int): Number of neurons in the first hidden layer.
            hidden_size_2 (int): Number of neurons in the second hidden layer.
            dropout_rate (float): The dropout probability.
        """
        super(ETF_MLP, self).__init__()
        
        # --- Layer Definitions ---
        self.layer_1 = nn.Linear(input_size, hidden_size_1)
        self.bn_1 = nn.BatchNorm1d(hidden_size_1)
        
        self.layer_2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.bn_2 = nn.BatchNorm1d(hidden_size_2)
        
        self.output_layer = nn.Linear(hidden_size_2, 1)
        
        # --- Activation and Regularization ---
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)
        
    def forward(self, x):
        """ The forward pass of the model. """
        # First hidden layer
        x = self.layer_1(x)
        x = self.bn_1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Second hidden layer
        x = self.layer_2(x)
        x = self.bn_2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Output layer with sigmoid for binary classification
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the model to test
input_features = X_train.shape[1]
model_mlp = ETF_MLP(input_size=input_features)
print("MLP Model Architecture:")
print(model_mlp)

MLP Model Architecture:
ETF_MLP(
  (layer_1): Linear(in_features=32, out_features=128, bias=True)
  (bn_1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer_2): Linear(in_features=128, out_features=64, bias=True)
  (bn_2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)


In [9]:
# Step 3: Manual MLP Training and Evaluation

# --- Configuration ---
INPUT_SIZE = X_train.shape[1]
LEARNING_RATE = 0.001
EPOCHS = 50

# --- Model, Loss, Optimizer (Demonstrates 5.2, 5.3) ---
model_mlp = ETF_MLP(input_size=INPUT_SIZE, dropout_rate=0.4)
criterion = nn.BCELoss() # Binary Cross Entropy Loss for binary classification
optimizer = torch.optim.Adam(model_mlp.parameters(), lr=LEARNING_RATE) # Adam Optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS) # LR Schedule

# --- MLflow Logging ---
with mlflow.start_run(run_name="MLP_Manual_Baseline") as run:
    mlflow.log_params({"learning_rate": LEARNING_RATE, "epochs": EPOCHS, "optimizer": "Adam"})
    
    # --- Training Loop ---
    for epoch in range(EPOCHS):
        model_mlp.train() # Set model to training mode
        for features, labels in train_loader:
            # Forward pass
            outputs = model_mlp(features)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Update learning rate
        scheduler.step()
        
        # --- Evaluation on Test Set ---
        model_mlp.eval() # Set model to evaluation mode
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for features, labels in test_loader:
                outputs = model_mlp(features)
                predicted = (outputs > 0.5).float()
                all_preds.extend(predicted.numpy())
                all_labels.extend(labels.numpy())
        
        # Calculate and log F1 score for the epoch
        f1 = f1_score(all_labels, all_preds)
        mlflow.log_metric("test_f1_score", f1, step=epoch)

    print(f"Final MLP F1 Score from manual run: {f1:.4f}")
    # Log the final model
    mlflow.pytorch.log_model(model_mlp, "mlp-model")

Final MLP F1 Score from manual run: 0.1261


In [10]:
# In notebooks/04_model_training_and_evaluation.ipynb, THE FINAL CELL

from mlflow.tracking import MlflowClient
import joblib

# Train and log the final champion model
with mlflow.start_run(run_name="XGBoost_Tuned_Champion") as run:
    print("--- Training and Logging Champion XGBoost Model ---")
    best_params = study.best_params
    
    mlflow.log_params(best_params)
    
    # Train the model
    model_xgb = xgb.XGBClassifier(**best_params, random_state=42)
    model_xgb.fit(X_train, y_train)
    
    # Evaluate and log metrics
    y_pred_xgb = model_xgb.predict(X_test)
    f1 = f1_score(y_test, y_pred_xgb)
    mlflow.log_metric("f1_score", f1)
    print(f"Champion XGBoost F1 Score: {f1:.4f}")

    # --- Log the Model, SHAP Assets, and Promote ---
    
    # 1. Log the model itself. This will create a new version.
    model_info = mlflow.xgboost.log_model(
        xgb_model=model_xgb,
        artifact_path="xgb-model",
        registered_model_name=MODEL_NAME
    )

    # 2. Find the new model version using the run_id from the logged model
    client = MlflowClient()
    run_id = model_info.run_id
    model_versions = client.search_model_versions(f"run_id='{run_id}'")
    new_version = model_versions[0].version # The first result should be our new version
    print(f"Model registered as '{MODEL_NAME}' version {new_version}.")

    # 3. Log SHAP assets for the dashboard
    explainer = shap.TreeExplainer(model_xgb)
    joblib.dump(explainer, "explainer.joblib")
    X_test.to_parquet("X_test.parquet")
    mlflow.log_artifact("explainer.joblib", artifact_path="shap_assets")
    mlflow.log_artifact("X_test.parquet", artifact_path="shap_assets")
    print("SHAP assets logged.")
    
    # 4. Promote this new version to the "Production" stage
    print(f"\n--- Promoting Model Version {new_version} to Production ---")
    client.transition_model_version_stage(
        name=MODEL_NAME,
        version=new_version,
        stage="Production",
        archive_existing_versions=True  # Safely archive any old production model
    )
    print(f"Successfully promoted model version {new_version} to 'Production'.")

--- Training and Logging Champion XGBoost Model ---
Champion XGBoost F1 Score: 0.6112


C:\Users\dawso\Dev\Personal\AIGrind\mlops-etf-forecasting\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [14:06:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


Registered model 'etf-xgboost-predictor' already exists. Creating a new version of this model...
Created version '2' of model 'etf-xgboost-predictor'.
C:\Users\dawso\AppData\Local\Temp\ipykernel_14152\1282367007.py:49: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


Model registered as 'etf-xgboost-predictor' version 2.
SHAP assets logged.

--- Promoting Model Version 2 to Production ---
Successfully promoted model version 2 to 'Production'.
